In [1]:
from procesadoDatosDemandaString import procesar_datos_string as procesar_datos_string_demanda 
from procesadoDatosPrecioString import procesar_datos_string as procesar_datos_string_precio
import mlxtend
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sklearn import preprocessing
from reglasAsociacion import discretiza_con_kmeans

In [2]:
demanda = procesar_datos_string_demanda()
precio = procesar_datos_string_precio()

Archivo donde están los datos de la demanda (sin extensión): demanda
Archivo donde están los datos del precio (sin extensión): precio


In [3]:
# Función para cambiar los valores de las reglas y que sepamos
# qué elementos aparecen en ella

def lode(x):
    if x == 0:
        return "dk0"
    elif x == 1:
        return "dk1"
    elif x == 2:
        return "dk2"
    elif x == 3:
        return "dk3"
    elif x == 4:
        return "dk4"
    elif x == 5:
        return "dk5"
    
def lopr(x):
    if x == 0:
        return "pk0"
    elif x == 1:
        return "pk1"
    elif x == 2:
        return "pk2"
    elif x == 3:
        return "pk3"
    elif x == 4:
        return "pk4"
    elif x == 5:
        return "dk5"

# Para reglas con 24, 23, 2 y 1 hora antes

In [4]:
lista_precio = precio.values.tolist()
lista_precio = [x for y in lista_precio for x in y][24:]
columna_precio = pd.DataFrame(lista_precio)

lista_demanda = demanda.values.tolist()

lista_demanda_1 = [x for y in lista_demanda for x in y][23:26279]

columna_demanda_1 = pd.DataFrame(lista_demanda_1)

lista_demanda = demanda.values.tolist()

lista_demanda_2 = [x for y in lista_demanda for x in y][22:26278]

columna_demanda_2 = pd.DataFrame(lista_demanda_2)

lista_demanda = demanda.values.tolist()

lista_demanda_24 = [x for y in lista_demanda for x in y][:26256]

columna_demanda_24 = pd.DataFrame(lista_demanda_24)

lista_demanda = demanda.values.tolist()

lista_demanda_23 = [x for y in lista_demanda for x in y][1:26257]

columna_demanda_23 = pd.DataFrame(lista_demanda_23)


discretiza1 = discretiza_con_kmeans(columna_demanda_1, 3)
discretiza1 = discretiza1.values.tolist()
discretiza1 = [lode(x) for y in discretiza1 for x in y]
discretiza1 = pd.DataFrame(discretiza1)

discretiza2 = discretiza_con_kmeans(columna_demanda_2, 3)
discretiza2 = discretiza2.values.tolist()
discretiza2 = [lode(x) for y in discretiza2 for x in y]
discretiza2 = pd.DataFrame(discretiza2)

discretiza23 = discretiza_con_kmeans(columna_demanda_23, 3)
discretiza23 = discretiza23.values.tolist()
discretiza23 = [lode(x) for y in discretiza23 for x in y]
discretiza23 = pd.DataFrame(discretiza23)

discretiza24 = discretiza_con_kmeans(columna_demanda_24, 3)
discretiza24 = discretiza24.values.tolist()
discretiza24 = [lode(x) for y in discretiza24 for x in y]
discretiza24 = pd.DataFrame(discretiza24)

discretizaprecio = discretiza_con_kmeans(columna_precio, 3)
discretizaprecio = discretizaprecio.values.tolist()
discretizaprecio = [lopr(x) for y in discretizaprecio for x in y]
discretizaprecio = pd.DataFrame(discretizaprecio)

dataset2 = pd.concat([discretiza1,
                      discretiza2,
                      discretiza23,
                      discretiza24,
                      discretizaprecio],
                       axis = 1)

dataset2 = dataset2.values.tolist()

In [7]:
te = TransactionEncoder()
te_ary = te.fit(dataset2).transform(dataset2)
df = pd.DataFrame(te_ary, columns=te.columns_)
reglas = apriori(df, min_support=0.25, use_colnames=True)
reglas

,support,itemsets
0,0.419980,(dk0)
1,0.777727,(dk1)
2,0.315318,(dk2)
3,0.891187,(pk1)
4,0.298789,"(dk0, dk1)"
5,0.368106,"(pk1, dk0)"
6,0.703192,"(pk1, dk1)"
7,0.279593,"(pk1, dk2)"
8,0.268129,"(pk1, dk0, dk1)"


In [8]:
rules = association_rules(reglas, metric="confidence", min_threshold=0.5)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(dk0),(dk1),0.419980,0.777727,0.298789,0.711436,0.914763,-0.027841,0.770272
1,(dk0),(pk1),0.419980,0.891187,0.368106,0.876485,0.983503,-0.006174,0.880972
2,(pk1),(dk1),0.891187,0.777727,0.703192,0.789051,1.014560,0.010092,1.053680
3,(dk1),(pk1),0.777727,0.891187,0.703192,0.904163,1.014560,0.010092,1.135394
4,(dk2),(pk1),0.315318,0.891187,0.279593,0.886701,0.994967,-0.001414,0.960410
5,"(pk1, dk0)",(dk1),0.368106,0.777727,0.268129,0.728401,0.936577,-0.018157,0.818388
6,"(dk0, dk1)",(pk1),0.298789,0.891187,0.268129,0.897387,1.006957,0.001853,1.060422
7,(dk0),"(pk1, dk1)",0.419980,0.703192,0.268129,0.638433,0.907907,-0.027197,0.820894
